In [1]:
!git clone https://github.com/mansasha21/QuartzNet5x5-CTC.git


Cloning into 'QuartzNet5x5-CTC'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 3), reused 20 (delta 1), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [2]:
from google.colab import drive
drive.mount('./gdrive')

Mounted at ./gdrive


In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import tarfile
my_tar = tarfile.open('/content/gdrive/MyDrive/asr_golos_data.tar')
my_tar.extractall('/content/QuartzNet5x5-CTC/data') # specify which folder to extract to
my_tar.close()

In [1]:
cd QuartzNet5x5-CTC/

/content/QuartzNet5x5-CTC


In [6]:
!pip install -r requirements_colab.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 708 kB 5.2 MB/s 
     |████████████████████████████████| 151 kB 69.4 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 529 kB 62.1 MB/s 
     |████████████████████████████████| 117 kB 69.5 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=16ffc982059b9d9575bf3c7257d31ee994640aef9c2dac8ef1cade55f878a4e0
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


In [6]:
import omegaconf
import torch
import pytorch_lightning as pl

from src.model import QuartzNetCTC, logger


conf = omegaconf.OmegaConf.load(open('/content/QuartzNet5x5-CTC/conf/quarznet_5x5_ru.yaml', 'r'))
model = QuartzNetCTC(conf)

ckpt = torch.load(conf.model.init_weights, map_location="cpu")
model.load_state_dict(ckpt["state_dict"])
print("successful load initial weights")

trainer = pl.Trainer(
    logger=pl.loggers.TensorBoardLogger(save_dir="logs"), **conf.trainer
)

successful load initial weights


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [7]:
res = trainer.validate(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            24.684438705444336
         val_wer            0.5935337543487549
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [16]:
conf = omegaconf.OmegaConf.load(open('/content/QuartzNet5x5-CTC/conf/quarznet_5x5_ru.yaml', 'r'))
model = QuartzNetCTC(conf)

ckpt = torch.load(conf.model.init_weights, map_location="cpu")
model.load_state_dict(ckpt["state_dict"])
print("successful load initial weights")

trainer = pl.Trainer(
    logger=pl.loggers.TensorBoardLogger(save_dir="logs"), **conf.trainer
)
trainer.fit(model)

successful load initial weights


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (WER). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:53: LightningDeprecationWarning: Setting `Trainer(resume_from_

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=12000` reached.


In [17]:
res = trainer.validate(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            20.144500732421875
         val_wer            0.5709854364395142
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Результаты дообучения
*   val_loss 24.6 -> 20.1
*   val_wer 0.59 -> 0.57

Да это немного, но это честная работа

-------------------
Я боялся, что колаб скажет мне покеда)

 